# Illumina 16S amplicon analysis with DADA2

In [1]:
library(dada2); packageVersion("dada2")
library(ShortRead); packageVersion("ShortRead")
library(ggplot2); packageVersion("ggplot2")

Loading required package: Rcpp


[1] ‘1.1.8’

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min

Loading required package: BiocParallel
Loading required package: Biostrings
Loading required package: S4Vectors
Loading required pack

[1] ‘1.32.0’

[1] ‘2.1.0’

## Input file handling

### This will be the root path for all of the FASTQ files

In [ ]:
path <- "data/itags/"

### Read in the sample to filename map

In [32]:
sample.map <- read.table("1.create_sample_map.txt", header=F, sep='\t', col.names=c("sample", "filename", "forward", "reverse"),  colClasses = "character")
sample.map$barcode <- sapply(strsplit(sample.map$filename, ".", fixed=T), `[`, 4)
head(sample.map)

sample,filename,forward,reverse,barcode
1-HH0420b,10759.1.176686.TAAGGCCTATC.fastq.gz,10759.1.176686.TAAGGCCTATC.forward.fastq.gz,10759.1.176686.TAAGGCCTATC.reverse.fastq.gz,TAAGGCCTATC
2-HH0424b,10759.1.176686.TTGTCGCACAA.fastq.gz,10759.1.176686.TTGTCGCACAA.forward.fastq.gz,10759.1.176686.TTGTCGCACAA.reverse.fastq.gz,TTGTCGCACAA
3-HH0427b,10759.1.176686.AACTGTGCGTA.fastq.gz,10759.1.176686.AACTGTGCGTA.forward.fastq.gz,10759.1.176686.AACTGTGCGTA.reverse.fastq.gz,AACTGTGCGTA
4-HH0429b,10759.1.176686.TGCGAACGTAT.fastq.gz,10759.1.176686.TGCGAACGTAT.forward.fastq.gz,10759.1.176686.TGCGAACGTAT.reverse.fastq.gz,TGCGAACGTAT
5-HH0501b,10759.1.176686.CATCAGCGTGT.fastq.gz,10759.1.176686.CATCAGCGTGT.forward.fastq.gz,10759.1.176686.CATCAGCGTGT.reverse.fastq.gz,CATCAGCGTGT
6-HH0503b,10759.1.176686.TTCAACAGGTG.fastq.gz,10759.1.176686.TTCAACAGGTG.forward.fastq.gz,10759.1.176686.TTCAACAGGTG.reverse.fastq.gz,TTCAACAGGTG


### Create matched lists of the forward and reverse file names

In [33]:
fastqs <- fns[grepl(".fastq.gz", fns)]
fastqs <- sort(fastqs) # Sort ensures forward/reverse reads are in same order
fnFs <- fastqs[grepl("forward", fastqs)] # Just the forward read files
fnRs <- fastqs[grepl("reverse", fastqs)] # Just the reverse read files
head(fnFs)
head(fnRs)

[1] "10759.1.176686.AACAGGTTCGC.forward.fastq.gz"
[2] "10759.1.176686.AACTGTGCGTA.forward.fastq.gz"
[3] "10759.1.176686.AAGTGGACTCT.forward.fastq.gz"
[4] "10759.1.176686.ACACGGTGTCT.forward.fastq.gz"
[5] "10759.1.176686.ACATCTTGACG.forward.fastq.gz"
[6] "10759.1.176686.ACGACGTCTTA.forward.fastq.gz"

[1] "10759.1.176686.AACAGGTTCGC.reverse.fastq.gz"
[2] "10759.1.176686.AACTGTGCGTA.reverse.fastq.gz"
[3] "10759.1.176686.AAGTGGACTCT.reverse.fastq.gz"
[4] "10759.1.176686.ACACGGTGTCT.reverse.fastq.gz"
[5] "10759.1.176686.ACATCTTGACG.reverse.fastq.gz"
[6] "10759.1.176686.ACGACGTCTTA.reverse.fastq.gz"

## Quality control plots

### Forward reads

In [ ]:
for(val in 1:length(fnFs)) {
    plotQualityProfile(paste0(path, fnFs[[1]]))
}

### Reverse reads

In [ ]:
for(val in 1:length(fnRs)) {
    plotQualityProfile(paste0(path, fnRs[[1]]))
}

## Perform filtering and trimming

We’ll use standard filtering parameters: maxN=0 (DADA2 requires no Ns), truncQ=2 (quality score 2 in Illumina means “stop using this read”) and maxEE=2. The maxEE parameter sets the maximum number of “expected errors” allowed in a read, which is a better filter than simply averaging quality scores. We use the fastqPairedFilter function to jointly filter the forward and reverse reads.

### Filter the forward and reverse reads:

Start by creating a directory

In [30]:
filt_path <- paste0(path, "filtered")
if(!file_test("-d", filt_path)) dir.create(filt_path)
filtFs <- file.path(filt_path, paste0(sample.map$sample, "_F_filt.fastq.gz"))
filtRs <- file.path(filt_path, paste0(sample.map$sample, "_R_filt.fastq.gz"))

Do the actual trimming

In [21]:
for(i in seq_along(fnFs)) {
  fastqPairedFilter(c(fnFs[i], fnRs[i]), c(filtFs[i], filtRs[i]),
                    trimLeft=c(10, 10), truncLen=c(240,160), 
                    maxN=0, maxEE=2, truncQ=2, 
                    compress=TRUE, verbose=TRUE)
}

ERROR: Error in parse(text = x, srcfile = src): <text>:1:27: unexpected string constant
1: filt_path <- data/filtered"
2: if(!file_test("
                             ^
